In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd 
import requests
import datetime as datetime
import time
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains 

option = Options()
option.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),options = option)
#actions = ActionChains(driver)
#link equal to daily covid data 
link = "https://covid.cdc.gov/covid-data-tracker/#trends_totalandratecases"
driver.get(link)
#let driver get link, takes time to load everything in
time.sleep(10)
#click on drop down menu and first option in drop down menu for states 
driver.find_element_by_xpath("/html/body/main/div/div[1]/div/div/div[1]/div[1]/div/div[1]/div[2]/i").click()
driver.find_element_by_xpath("/html/body/main/div/div[1]/div/div/div[1]/div[1]/div/div[1]/div[2]/div[2]/div[2]").click()
#parse through the html
source = driver.page_source 
#information needed is embedded within several tags so we go through each tag to get the info we want
soup = BeautifulSoup(source, 'html.parser')
soup_2 =  BeautifulSoup(source, 'html.parser')
info = soup.find_all('table', class_= 'expanded-data-table')
info_2 = soup.find_all('tr')
info_3 = soup.find_all('td')
info_3 = list(info_3)

#scrape the list of dates available in the data
for i in range(0,len(info_3)):
    info_3[i] = info_3[i]['aria-label']
date = []
for i in range(0,len(info_3),3):
    holder = info_3[i].split(' ')
    date.append(holder[1]+' ' + holder[2]+ ' '+ holder[3])

#create dataframe for everything scrapped starting with Date as the first column
covid_data = list(zip(date))
covid_data = pd.DataFrame(covid_data, columns = ['Date'])

#scrape all the state names as the CDC data includes NYC as separate as well as territories
state_names = []
states = soup_2.find_all('div', class_ = 'item')
for i in range(0,len(states)):
    state_names.append(states[i].text)

#chromedriver to go to the cdc website 
option = Options()
option.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),options = option)
link = "https://covid.cdc.gov/covid-data-tracker/#trends_totalandratecases"
driver.get(link)
time.sleep(10)
#click on the dropdown menu again
driver.find_element_by_xpath("/html/body/main/div/div[1]/div/div/div[1]/div[1]/div/div[1]/div[2]/i").click()
time.sleep(5)
#get total count of number of items in dropdown menu
total_count = driver.find_elements_by_xpath("//div[@class='menu transition visible']/div")
#loop through drop down menu one by one
#reason for this is that the table on the site is updated as you click through each option
#each option is clicked, the data of interest is scrapped, and next item is clicked and so on
for i in range(len(total_count)):

    path = "//div[@class='menu transition visible']/div[{}]".format(i+1)

    driver.find_element_by_xpath(path).click()
    ####scrape here
    source = driver.page_source 
    soup = BeautifulSoup(source, 'html.parser')
    info = soup.find_all('table', class_= 'expanded-data-table')
    info_2 = soup.find_all('tr')
    info_3 = soup.find_all('td')
    info_3 = list(info_3)
    #loop through to remove the aria-label in front of each piece of data we want
    for x in range(0,len(info_3)):
        info_3[x] = info_3[x]['aria-label']
    daily_case = []
    case_rate = []

    #find the daily cases and well as cases per 100,000
    for x in range(1, len(info_3),3):
        holder = info_3[x].split(' ')
        daily_case.append(holder[-1])

    for x in range(2, len(info_3),3):
        holder = info_3[x].split(' ')
        case_rate.append(holder[-1])
    #scrapped data added to overall dataframe including name of territory/state as well as what the data represents
    
    covid_data[state_names[i] + ' Total Cases'] = list(daily_case)
    covid_data[state_names[i] + ' Cases Per 100,000'] = list(case_rate)
    print(state_names[i]+' read in for cases')
    #click drop down menu to reset 
    driver.find_element_by_xpath("/html/body/main/div/div[1]/div/div/div[1]/div[1]/div/div[1]/div[2]/i").click()
    time.sleep(3)
    
#below is the same process but for COVID death data whereas above was just for cases
option = Options()
option.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),options = option)
#actions = ActionChains(driver)
#link equal to daily covid data 
link = "https://covid.cdc.gov/covid-data-tracker/#trends_totalandratedeaths"
driver.get(link)
#let driver get link, takes time to load everything in
time.sleep(10)

driver.find_element_by_xpath("/html/body/main/div/div[1]/div/div/div[1]/div[1]/div/div[1]/div[2]/i").click()
driver.find_element_by_xpath("/html/body/main/div/div[1]/div/div/div[1]/div[1]/div/div[1]/div[2]/div[2]/div[2]").click()
#parse through the html
source = driver.page_source 
#information needed is embedded within several tags so we go through each tag to get the info we want
soup = BeautifulSoup(source, 'html.parser')
soup_2 =  BeautifulSoup(source, 'html.parser')
info = soup.find_all('table', class_= 'expanded-data-table')
info_2 = soup.find_all('tr')
info_3 = soup.find_all('td')
info_3 = list(info_3)

for i in range(0,len(info_3)):
    info_3[i] = info_3[i]['aria-label']
date = []
for i in range(0,len(info_3),3):
    holder = info_3[i].split(' ')
    date.append(holder[1]+' ' + holder[2]+ ' '+ holder[3])

covid_data_death = list(zip(date))
covid_data_death = pd.DataFrame(covid_data_death, columns = ['Date'])
    
state_names = []
states = soup_2.find_all('div', class_ = 'item')
for i in range(0,len(states)):
    state_names.append(states[i].text)

option = Options()
option.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),options = option)
link = "https://covid.cdc.gov/covid-data-tracker/#trends_totalandratedeaths"
driver.get(link)
time.sleep(10)
driver.find_element_by_xpath("/html/body/main/div/div[1]/div/div/div[1]/div[1]/div/div[1]/div[2]/i").click()
time.sleep(5)
total_count = driver.find_elements_by_xpath("//div[@class='menu transition visible']/div")
for i in range(len(total_count)):
   
    path = "//div[@class='menu transition visible']/div[{}]".format(i+1)

    driver.find_element_by_xpath(path).click()
    ####scrape here
    source = driver.page_source 
    soup = BeautifulSoup(source, 'html.parser')
    info = soup.find_all('table', class_= 'expanded-data-table')
    info_2 = soup.find_all('tr')
    info_3 = soup.find_all('td')
    info_3 = list(info_3)
    #loop through to remove the aria-label in front of each piece of data we want
    for x in range(0,len(info_3)):
        info_3[x] = info_3[x]['aria-label']
    daily_case = []
    case_rate = []

   #get daily deaths as well as deaths per 100,000
    for x in range(1, len(info_3),3):
        holder = info_3[x].split(' ')
        daily_case.append(holder[-1])

    for x in range(2, len(info_3),3):
        holder = info_3[x].split(' ')
        case_rate.append(holder[-1])

    covid_data_death[state_names[i] + ' Total Deaths'] = list(daily_case)
    covid_data_death[state_names[i] + ' Deaths Per 100,000'] = list(case_rate)
    print(state_names[i]+' read in for deaths')
    driver.find_element_by_xpath("/html/body/main/div/div[1]/div/div/div[1]/div[1]/div/div[1]/div[2]/i").click()
    time.sleep(3)
covid_data.to_csv('covid_state_cases.csv')
covid_data_death.to_csv('covid_state_deaths.csv')